In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from PIL import Image
import pytesseract
import time
import subprocess
import os

In [2]:
import ctypes
user32 = ctypes.windll.user32
screensize = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

In [3]:
def get_captcha(driver, element, path):
    # now that we have the preliminary stuff out of the way time to get that image :D
    location = element.location

    size = element.size

    # saves screenshot of entire page
    driver.save_screenshot(path)

    # uses PIL library to open image in memory
    image = Image.open(path)

    left = location['x']
    top = location['y'] 
    right = location['x'] + size['width']
    bottom = location['y'] + size['height']

    image = image.crop((left, top, right, bottom))  # defines crop points
    image.save(path, 'png')  # saves new cropped image

In [4]:
def screen_resolution(w,h,screen):
    directory = os.getcwd()
    with open(os.path.join(directory, 'screen_resolution_to_{}_{}.bat'.format(w,h)), 'w') as OPATH:
        OPATH.writelines(['ChangeScreenResolution.exe /w={} /h={} /d={} \n'.format(w,h,screen),
                          '\n'
                          'exit'])
    subprocess.call([os.getcwd() +  '\\screen_resolution_to_{}_{}.bat'.format(w,h)])
    OPATH.close()

In [5]:
def click_button(browser ,button_text):
    try:
        button = browser.find_element_by_link_text(button_text)
        button.click()
    except:
        pass

In [6]:
browser = webdriver.Chrome(r'C:\Users\g701295\webscrapping_rahul\chromedriver_win32\chromedriver.exe')
browser.get('http://igrmaharashtra.gov.in/frmHome.aspx')
browser.maximize_window()
time.sleep(2)

browser.find_element_by_xpath("/html/body/div/div/form/table[2]/tbody/tr/td/table[1]/tbody/tr[4]/td/table/tbody/tr/td[1]/input[2]").click()
time.sleep(5)

browser.find_element_by_link_text('English').click()
time.sleep(5)

browser.find_element_by_xpath("//span[text()='e-Search']").click()
browser.find_element_by_link_text('Free Process').click()
time.sleep(5)

child = browser.window_handles[1]      
browser.switch_to.window(child)
time.sleep(5)

In [7]:
def fill_page(browser, year, district, first_four_letter_of_village, village_name, property_number):

    browser.find_element_by_xpath("//*[@id='ddlFromYear']").send_keys(str(year))

    district_dropdown = browser.find_element_by_xpath("//*[@id='ddlDistrict']")
    district_select = Select(district_dropdown)
    district_select.select_by_value(str(district))
    time.sleep(5)

    vill_text_area = browser.find_element_by_id('txtAreaName')
    vill_text_area.send_keys(first_four_letter_of_village)
    time.sleep(1)

    browser.find_element_by_xpath("//*[@id='ddlareaname']").click()
    time.sleep(5)

    village_dropdown = browser.find_element_by_xpath("//*[@id='ddlareaname']")
    village_select = Select(village_dropdown)
    village_select.select_by_value(village_name)

    property_text_area = browser.find_element_by_id('txtAttributeValue')
    property_text_area.send_keys(str(property_number))

    w = 1920
    h = 1080
    screen_resolution(w,h,1)

    # browser.save_screenshot("screenshot.png")
    img = browser.find_element_by_xpath("//*[@id='imgCaptcha']")
    get_captcha(browser, img, "captcha.png")
    time.sleep(5)

    im = Image.open("captcha.png")
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\g701295\AppData\Local\Tesseract-OCR\tesseract.exe'
    captcha_text = pytesseract.image_to_string(im, lang='eng')

    vill_text_area = browser.find_element_by_id('txtImg')
    vill_text_area.send_keys(captcha_text)

    browser.find_element_by_xpath("//*[@id='btnSearch']").click()

    w = screensize[0]
    h = screensize[1]
    screen_resolution(w,h,1)
    time.sleep(5)
    return browser

In [28]:
pagetab=2
page=11

In [30]:
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
currenttab = pages.find_elements_by_tag_name('td')[8].text
if (((pagetab==1)&(page==10))|((pagetab>1)&(page==11))):         
    page_clicked = pages.find_elements_by_tag_name('td')[8].text
    while (page_clicked==currenttab):  
        print(page_clicked,currenttab)
        try:                
            #clicking on a given page
            xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
            browser.find_element_by_xpath(xpath).click()
            time.sleep(3)  
            pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
            page_clicked = pages.find_elements_by_tag_name('td')[8].text
        except:
            print('not_clicked')

18 18


In [8]:
browser = fill_page(browser, 2018, 31, 'Kole', 'Kolekalyan', 4207)

In [10]:
# %%timeit
table_end = False
pagetab = 1

#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))
row_list = []
col_list = []

In [ ]:
while not table_end:
    if (((pagetab==1)&(count_pages<11))|((pagetab>1)&(count_pages<12))):
        table_end = True     
        
    #Number fo columns
    xpath  = "//*[@id='RegistrationGrid']/tbody/tr[2]"
    table = browser.find_element_by_xpath(xpath)
    column_count = len(table.find_elements_by_tag_name('td'))  
    
    #Counting the number of pages that are present for clicking on the given page
    pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
    count_pages = len(pages.find_elements_by_tag_name('td'))
    
    for page in range(count_pages-10,count_pages):
        page_clicked = False
#         print((page>1),(page>2),pagetab==1,((page>1)&(pagetab==1)),((page>2)&(pagetab>1)),((page>1)&(pagetab==1))|((page>2)&(pagetab>1)))
        if ((page>1)&(pagetab==1))|((page>2)&(pagetab>1)):
            while not page_clicked:                
                try:
                    #clicking on a given page
                    xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page) + "]/a"
                    browser.find_element_by_xpath(xpath).click()
                    time.sleep(3)
                except:
                    page_clicked = True
        else:
            print('page1 of tab 1 or page2 of other tab')
        print('inner',11, column_count,page-1,pagetab,page_clicked)   
        
        #Number fo columns
        table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody")
        row_count = len(table.find_elements_by_tag_name('tr'))-2
        
        time.sleep(3)
        for row in range(2,row_count+1):            
            for col in range(1, column_count+1):
                xpath = "//*[@id='RegistrationGrid']/tbody/tr[" + str(row) + "]/td[" + str(col) + "]"
                col_list.append(browser.find_element_by_xpath(xpath).text)
                
    #Counting the number of pages that are present for clicking on the given page
    pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
    currenttab = pages.find_elements_by_tag_name('td')[8].text
    click_count = 0
    if (((pagetab==1)&(page==10))|((pagetab>1)&(page==11))):         
        page_clicked = pages.find_elements_by_tag_name('td')[8].text
        while (page_clicked==currenttab)|(click_count>5):  
            print(page_clicked,currenttab)
            try:                
                #clicking on a given page
                xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
                browser.find_element_by_xpath(xpath).click()
                time.sleep(3)  
                click_count = click_count + 1
                pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
                page_clicked = pages.find_elements_by_tag_name('td')[8].text
            except:
                print('not_clicked')
#         if (click_count>5):
#             browser = fill_page(browser, year, district, first_four_letter_of_village, village_name, property_number)
        #Counting the number of pages that are present for clicking on the given page
        pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
        count_pages = len(pages.find_elements_by_tag_name('td'))
        pagetab = pagetab + 1

page1 of tab 1 or page2 of other tab
inner 11 10 0 1 False
inner 11 10 1 1 True
inner 11 10 2 1 True
inner 11 10 3 1 True
inner 11 10 4 1 True
inner 11 10 5 1 True
inner 11 10 6 1 True
inner 11 10 7 1 True
inner 11 10 8 1 True
inner 11 10 9 1 True
9 9
page1 of tab 1 or page2 of other tab
inner 11 10 1 2 False
inner 11 10 2 2 True
inner 11 10 3 2 True
inner 11 10 4 2 True
inner 11 10 5 2 True
inner 11 10 6 2 True
inner 11 10 7 2 True
inner 11 10 8 2 True
inner 11 10 9 2 True
inner 11 10 10 2 True
18 18
page1 of tab 1 or page2 of other tab
inner 11 10 1 3 False
inner 11 10 2 3 True
inner 11 10 3 3 True
inner 11 10 4 3 True
inner 11 10 5 3 True
inner 11 10 6 3 True
inner 11 10 7 3 True
inner 11 10 8 3 True
inner 11 10 9 3 True
inner 11 10 10 3 True
28 28
page1 of tab 1 or page2 of other tab
inner 11 10 1 4 False
inner 11 10 2 4 True
inner 11 10 3 4 True
inner 11 10 4 4 True
inner 11 10 5 4 True
inner 11 10 6 4 True
inner 11 10 7 4 True
inner 11 10 8 4 True
inner 11 10 9 4 True
inner 11 10

In [13]:
currenttab

'9'

9 9


In [ ]:
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")

In [4]:
#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))

NameError: name 'browser' is not defined

In [3]:
pages

NameError: name 'pages' is not defined

In [2]:
pages.find_elements_by_tag_name('td')[8].text

NameError: name 'pages' is not defined

In [ ]:
<a href="javascript:__doPostBack('RegistrationGrid','Page$85')" style="color:Black;">85</a>

In [16]:
browser.find_element_by_link_text("javascript:__doPostBack('RegistrationGrid','Page$85')").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"javascript:__doPostBack('RegistrationGrid','Page$85')"}
  (Session info: chrome=76.0.3809.100)


In [15]:
len(col_list)/11

459.09090909090907

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[12]/td/table/tbody/tr/td[6]/a

In [40]:
(pagetab==1&page==10)|(pagetab>1&page==11)

False

In [29]:
xpath = "//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[" + str(page+1) + "]/a"
browser.find_element_by_xpath(xpath).click()
time.sleep(8)   

In [44]:
pagetab==1

True

In [34]:
page==10

True

In [36]:
((pagetab==1)&(page==10))

True

In [26]:
table_end

True

In [31]:
print(page,pagetab)

10 1


In [93]:
row_list

[['10052',
  'गहाणखत',
  '14/08/2018',
  'सह दु.नि. अंधेरी 1',
  '{"सेंट्रल बँक ऑफ इंडिया भारत डायमन्ड बोर्स ब्रांचतर्फे असिसस्टंट. मॅनिजरसुमा नायर"}',
  '{"आर चिमनलाल अँड कोतर्फे पार्टनरअतुल रजनिकुमार झवेरी"}',
  'सदनिका नं: ऑफिस डी सी 5150, माळा नं: 5 फ़्लोर डी टावर, इमारतीचे नाव: भारत डायमन्ड बोर्स कॉमप्लेक्स सेंट्रल विंग, ब्लॉक नं: कार पार्किग स्पन्स नं.डी-761 बांदरा ईस्ट, रोड नं: बीकेएसी रोड',
  '322',
  '4',
  ''],
 ['10383',
  'सप्लीमेंट्री अँग्रीमेंट',
  '24/08/2018',
  'सह दु.नि. अंधेरी 1',
  '{"स्टारलाईट सिस्टम्स आय एल.एल.पी तर्फे प्राधिकृत व्यक्ती स्वेता गुप्ता यांच्या तर्फे मुखत्यार कमलेश अगरवाल--","स्टारलाईट सिस्टम्स आय एल.एल.पी तर्फे प्राधिकृत व्यक्ती मारिया अविताम्पिल्ली यांच्या तर्फे मुखत्यार कमलेश अगरवाल--"}',
  '{"लक्ष्मण नरसिंम्हन--"}',
  'सदनिका नं: 503,बी विंग, माळा नं: 5 वा मजला, इमारतीचे नाव: सिग्नेचर आयलंड, ब्लॉक नं: बांद्रा पूर्व,मुंबई-400051, रोड नं: जी ब्लॉक,बांद्रा कुर्ला कॉम्प्लेक्स, इतर माहिती: दस्तात नमूद केल्याप्रमाणे. दस्त क्रमांक बदर-4/1979/2011 या दस्त

In [92]:
len(row_list)

28

In [30]:
#Number fo columns
table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[10]")
column_count = len(table.find_elements_by_tag_name('td'))

#Number fo columns
table = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody")
row_count = len(table.find_elements_by_tag_name('tr'))-2

In [ ]:
for row, col in browser.find_elements_by_xpath('//*[@id="tblmainn"]/tbody/tr'):
    tds = tr.find_elements_by_tag_name('col')
    print([td.text for td in tds])

In [31]:
# child = browser.window_handles[1]      
# browser.switch_to.window(child) 

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[12]/td/table/tbody/tr/td[12]/a

In [67]:
#Counting the number of pages that are present for clicking on the given page
pages = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr")
count_pages = len(pages.find_elements_by_tag_name('td'))

In [66]:
#clicking on a given page
browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[12]/td/table/tbody/tr/td[11]/a").click()

In [192]:
col_len = len(browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody").find_elements_by_tag_name('td'))

In [152]:
browser.find_element_by_xpath("//*[@id='tblmain']/tbody/tr[3]/td/div/table").text

'DocNo DName RDate SROName Seller Name Purchaser Name Property Description SROCode Status IndexII\n7485 सेल डीड 15/06/2018 सह दु.नि. अंधेरी 1 {"येशा डायमंड्स तर्फे भागिदार नितिनकांतिलालरावल","येशा डायमंड्स तर्फे भागिदार जिगरनितिनरावल"} {साधनानितिनकुमाररावल,मितमनोजमेहता} सदनिका नं: ऑफिस प्रीमाईसीस नं.एफडब्लू-2110,एफ टावर,वेस्ट विंग, माळा नं: 2 रा माळा, इमारतीचे नाव: भारत डायमंड बुर्स कॉमप्लेक्स, ब्लॉक नं: जी ब्लॉक,बांदरा कुर्ला कॉमप्लेक्स, रोड नं: बांदरा पूर्व,मुंबई-400051 322 4\n4932 करारनामा 21/04/2018 सह दु.नि. अंधेरी 1 {"स्टारलाईट सिस्टम्स आय एल एल पी तर्फे प्राधिकृत व्यक्ति स्वेता गुप्ता यांच्या तर्फे मुखत्यार गोपाल क्रीशन अगरवाल--","स्टारलाईट सिस्टम्स आय एल एल पी तर्फे प्राधिकृत व्यक्ति मेहुल तोलिया यांच्या तर्फे मुखत्यार गोपाल क्रीशन अगरवाल--"} {"एक्सपांडेड पॉलीमर सिस्टम्स प्रायवेट लीमीटेड तर्फे संचालक मुकेश एस.भूता--","एक्सपांडेड पॉलीमर सिस्टम्स प्रायवेट लीमीटेड तर्फे संचालक राजेन मेहता--","मुकेश शांतीलाल.भूता--"} सदनिका नं: युनिट नं.204, माळा नं: दुसरा मजला, इमारतीचे नाव: सिग्न

In [90]:
# pd.read_html('https://freesearchigrservice.maharashtra.gov.in/')

In [ ]:
//*[@id="tblmain"]/tbody/tr[3]/td/div/table

In [53]:
elem.text.split('\n')

['DocNo DName RDate SROName Seller Name Purchaser Name Property Description SROCode Status IndexII',
 '7485 सेल डीड 15/06/2018 सह दु.नि. अंधेरी 1 {"येशा डायमंड्स तर्फे भागिदार नितिनकांतिलालरावल","येशा डायमंड्स तर्फे भागिदार जिगरनितिनरावल"} {साधनानितिनकुमाररावल,मितमनोजमेहता} सदनिका नं: ऑफिस प्रीमाईसीस नं.एफडब्लू-2110,एफ टावर,वेस्ट विंग, माळा नं: 2 रा माळा, इमारतीचे नाव: भारत डायमंड बुर्स कॉमप्लेक्स, ब्लॉक नं: जी ब्लॉक,बांदरा कुर्ला कॉमप्लेक्स, रोड नं: बांदरा पूर्व,मुंबई-400051 322 4',
 '4932 करारनामा 21/04/2018 सह दु.नि. अंधेरी 1 {"स्टारलाईट सिस्टम्स आय एल एल पी तर्फे प्राधिकृत व्यक्ति स्वेता गुप्ता यांच्या तर्फे मुखत्यार गोपाल क्रीशन अगरवाल--","स्टारलाईट सिस्टम्स आय एल एल पी तर्फे प्राधिकृत व्यक्ति मेहुल तोलिया यांच्या तर्फे मुखत्यार गोपाल क्रीशन अगरवाल--"} {"एक्सपांडेड पॉलीमर सिस्टम्स प्रायवेट लीमीटेड तर्फे संचालक मुकेश एस.भूता--","एक्सपांडेड पॉलीमर सिस्टम्स प्रायवेट लीमीटेड तर्फे संचालक राजेन मेहता--","मुकेश शांतीलाल.भूता--"} सदनिका नं: युनिट नं.204, माळा नं: दुसरा मजला, इमारतीचे नाव

In [49]:
a = pd.DataFrame(elem.text.split('\n'))

In [50]:
a

,0
0,DocNo DName RDate SROName Seller Name Purchase...
1,"7485 सेल डीड 15/06/2018 सह दु.नि. अंधेरी 1 {""य..."
2,"4932 करारनामा 21/04/2018 सह दु.नि. अंधेरी 1 {""..."
3,6259 अभिहस्तांतरणपत्र 19/05/2018 सह दु.नि. अंध...
4,5004 65-चुक दुरुस्ती पत्र 23/04/2018 सह दु.नि....
5,11616 36-अ-लिव्ह अॅड लायसन्सेस 29/09/2018 सह द...
6,1173 अभिहस्तांतरणपत्र 29/01/2018 सह दु.नि. अंध...
7,242 हक्कलेखनिक्षेप दस्त 06/01/2018 सह दु.नि. अ...
8,"1494 सेल डीड 03/02/2018 सह दु.नि. अंधेरी 1 {""ज..."
9,332 36-अ-लिव्ह अॅड लायसन्सेस 09/01/2018 सह दु....


In [52]:
(str(a.iloc[2]))

'0    4932 करारनामा 21/04/2018 सह दु.नि. अंधेरी 1 {"...\nName: 2, dtype: object'

In [35]:
row = browser.find_element_by_xpath("//*[@id='RegistrationGrid']/tbody/tr[2]/td[7]") 

In [36]:
row.text

'सदनिका नं: ऑफिस प्रीमाईसीस नं.एफडब्लू-2110,एफ टावर,वेस्ट विंग, माळा नं: 2 रा माळा, इमारतीचे नाव: भारत डायमंड बुर्स कॉमप्लेक्स, ब्लॉक नं: जी ब्लॉक,बांदरा कुर्ला कॉमप्लेक्स, रोड नं: बांदरा पूर्व,मुंबई-400051'

In [31]:
import pandas as pd

In [ ]:
for rowno in 
for  colno in range(0,9):


In [32]:
data = pd.DataFrame(columns=['DocNo','DName','RDate','SROName','Seller', 'Name','Purchaser', 'Name','Property',
                      'Description','SROCode','Status','IndexII'])

ValueError: DataFrame constructor not properly called!

In [24]:
//*[@id="RegistrationGrid"]

''

In [192]:
//*[@id="btnSearch"]

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[2]/td[7]/text()

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[3]/td[7]/text()

In [ ]:
//*[@id="RegistrationGrid"]/tbody/tr[2]/td[10]/input

In [194]:
//*[@id="RegistrationGrid"]/tbody/tr[3]/td[10]/input

'23667'